## IMPORT PACKAGES

In [ ]:
import pandas as pd
import sys
from matplotlib import pyplot as plt

sys.path.append('../../')
from utils import microarray_utils, clustering_utils, bokeh_ui_utils

In [ ]:
full_filtered_df = pd.read_csv('../microarray_probe_alignment_and_filtering/allgood_filt_agg_tidy_2021aligned_qc_rma_expression_full.csv')
full_filtered_df = full_filtered_df.rename(columns={'Unnamed: 0': 'TTHERM_ID'})
num_genes = full_filtered_df.shape[0]
num_genes

In [ ]:
full_filtered_df.sample(10)

In [ ]:
full_filtered_norm_df = microarray_utils.normalize_expression_per_gene(full_filtered_df)

# LEIDEN CLUSTERING

In [ ]:
phases = 'full'

In [ ]:
leiden_label_df_round_1, partition_stats, cluster_sizes, enriched_cluster_sizes =  clustering_utils.build_label_df(
    full_filtered_df, 
    # metric='minkowski_3.0', 
    metric='angular',
    n_neighbors=2, 
    resolution_param=0.001, 
    partition_type = 'EXP', 
    n_jobs = -1, 
    random_state=42
    )

In [ ]:
leiden_label_df_round_1_arranged = bokeh_ui_utils.arrange_modules(full_filtered_norm_df, leiden_label_df_round_1, phases) 
leiden_label_df_round_1_arranged_sorted = leiden_label_df_round_1_arranged.sort_values(by=['label', 'TTHERM_ID'], ascending=False)
leiden_label_df_round_1_arranged_sorted.to_csv('./test_nn3_leiden_label_df_round_1.csv', index=False)

In [ ]:
plt.hist(cluster_sizes, max(cluster_sizes))
plt.title('DISTRIBUTION OF ALL CLUSTER SIZES')
plt.xlabel('# genes in cluster')
plt.ylabel('# clusters')
plt.show()

In [ ]:
plt.hist(enriched_cluster_sizes, max(enriched_cluster_sizes))
plt.title('DISTRIBUTION OF ENRICHED CLUSTER SIZES')
plt.xlabel('# genes in cluster')
plt.ylabel('# clusters')
plt.show()

In [ ]:
key_lens = [len(k) for k in partition_stats.keys()]
max_key_len = max(key_lens)

for k, v in partition_stats.items():
    print(f'{" " * (max_key_len - len(k))}{k}:', v)

In [ ]:
# for MCL
def leiden_clustering_df_to_csv(df, export_file_path):
    current_module = list(df['label'].values)[0]

    new_cluster = True

    with open(export_file_path, 'w') as f:
        for idx, row in df.iterrows():
            if current_module != row['label']:
                current_module = row['label']
                f.write('\n')
                new_cluster =True

            elif not new_cluster:
                f.write('\t')
            
            f.write(row['TTHERM_ID'])
            new_cluster = False

        f.write('\n')
    
    print(export_file_path, 'successfully exported.')